In [1]:
import pandas as pd
import os
os.chdir('/Users/ronaldliu/Downloads/jiang_2023/Scripts_2')

import numpy as np
import pandas as pd
from Data import equity_data as eqd   # make sure Scripts_2 is on PYTHONPATH
from __future__ import print_function, division
import os
import os.path as op
import numpy as np
import pandas as pd
import time
from Data import dgp_config as dcf
from tqdm import tqdm

In [34]:

import pandas as pd
import numpy as np

file_path = os.path.join(dcf.CACHE_DIR, f"SSE.csv")
# ── 1.  Load & clean  ──────────────────────────────────────────────────────────
df = (
    pd.read_csv(file_path,  # or io.StringIO(your-string)
                parse_dates=["date"])
      .sort_values("date")                      # ascending chronology
)

# %-strings → decimal returns (e.g. "1.15%" → 0.0115)
df["ret"] = df["Ret"].str.rstrip("%").astype(float) / 100

# ── 2.  Daily log-return & cumulative-log-return  ──────────────────────────────
df["log_ret"]  = np.log1p(df["ret"])
df["cum_log_ret"] = df["log_ret"].cumsum()

# ── 4.  Period returns & “next-period” returns  ───────────────────────────────
freq = "week"
freq_col       = f"{freq}_ret"          # e.g. week_ret

period_ends = eqd.get_period_end_dates("week", "CN")
period_df   = df[df["date"].isin(period_ends)].copy()

# return for the period that ENDS at this row
period_df[freq_col] = (
    np.exp(period_df["cum_log_ret"] - period_df["cum_log_ret"].shift(1)) - 1
)

# --- keep only the two wanted columns ---
period_df = period_df[['date', 'week_ret']].copy()
period_df.columns = ["date", "Ret"]
# --- optional: make the left-hand index clean 0,1,2,… ---
period_df.reset_index(drop=True, inplace=True)

# Shift returns to align with the start of the period
period_df["Ret"] = period_df["Ret"].shift(-2)

print(period_df)

file_path = os.path.join(dcf.CACHE_DIR, f"SSE_{freq}.csv")
period_df.to_csv(file_path, index=False)

          date       Ret
0   2010-01-08 -0.029748
1   2010-01-15 -0.044501
2   2010-01-22 -0.016647
3   2010-01-29  0.026817
4   2010-02-05  0.011255
..         ...       ...
783 2025-04-30  0.007626
784 2025-05-09 -0.005740
785 2025-05-16 -0.000237
786 2025-05-23       NaN
787 2025-05-30       NaN

[788 rows x 2 columns]


In [ ]:
eqd.get_benchmark_returns("week", "CN")


Date
2010-01-08   -0.024826
2010-01-15    0.008864
2010-01-22   -0.029748
2010-01-29   -0.044501
2010-02-05   -0.016647
                ...   
2025-04-30   -0.004793
2025-05-09    0.019178
2025-05-16    0.007626
2025-05-23   -0.005740
2025-05-30   -0.000237
Name: Ret, Length: 788, dtype: float64

In [9]:
eqd.get_benchmark_returns("week", "US")

Date
1993-01-29    0.004923
1993-02-05    0.019307
1993-02-19   -0.020379
1993-02-26    0.010757
1993-03-05    0.016202
                ...   
2019-12-06    0.003427
2019-12-13    0.010645
2019-12-20    0.015208
2019-12-27    0.007579
2019-12-31    0.003259
Name: ewretx, Length: 1278, dtype: float64